In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

/home/iron/Install/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import re
def rm_tags(text):
    re_tag = re.compile(r'<[^>]+>')
    return re_tag.sub('', text)
import os
def read_files(filetype):
    path = "data/aclImdb/"
    file_list = []
    
    positive_path = path + filetype + "/pos/"
    for f in os.listdir(positive_path):
        file_list += [positive_path+f]
        
    negative_path = path + filetype + "/neg/"
    for f in os.listdir(negative_path):
        file_list += [negative_path+f]
    
    print('read', filetype, 'files:', len(file_list))
    
    all_labels = ([1] * 12500 + [0] * 12500)
    
    all_texts = []
    for fi in file_list:
        with open(fi, encoding='utf8') as file_input:
            all_texts += [rm_tags(" ".join(file_input.readlines()))]
            
    return all_labels, all_texts

In [3]:
y_train, train_text = read_files("train")

read train files: 25000


In [4]:
y_test, test_text = read_files("test")

read test files: 25000


In [63]:
token = Tokenizer(num_words=3800)
token.fit_on_texts(train_text)

In [64]:
x_train_seq = token.texts_to_sequences(train_text)
x_test_seq = token.texts_to_sequences(test_text)

In [65]:
x_train = sequence.pad_sequences(x_train_seq, maxlen=380)
x_test = sequence.pad_sequences(x_test_seq, maxlen=380)

In [66]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding

In [67]:
model = Sequential()

In [68]:
model.add(Embedding(output_dim=32, input_dim=3800, input_length=380))

In [69]:
model.add(Dropout(0.2))

In [70]:
model.add(Flatten())

In [71]:
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.35))

In [72]:
model.add(Dense(units=1, activation='sigmoid'))

In [73]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 380, 32)           121600    
_________________________________________________________________
dropout_3 (Dropout)          (None, 380, 32)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 12160)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               3113216   
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 3,235,073
Trainable params: 3,235,073
Non-trainable params: 0
_________________________________________________________________


In [74]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [76]:
train_history = model.fit(x_train, y_train, batch_size=100, epochs=10, verbose=2, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
 - 28s - loss: 0.3144 - acc: 0.8663 - val_loss: 0.4489 - val_acc: 0.8084
Epoch 2/10
 - 29s - loss: 0.1280 - acc: 0.9542 - val_loss: 0.4889 - val_acc: 0.8210
Epoch 3/10
 - 28s - loss: 0.0451 - acc: 0.9871 - val_loss: 0.6144 - val_acc: 0.8192
Epoch 4/10
 - 29s - loss: 0.0172 - acc: 0.9957 - val_loss: 1.0596 - val_acc: 0.7606
Epoch 5/10
 - 29s - loss: 0.0098 - acc: 0.9977 - val_loss: 1.0565 - val_acc: 0.7832
Epoch 6/10
 - 30s - loss: 0.0074 - acc: 0.9981 - val_loss: 0.9000 - val_acc: 0.8224
Epoch 7/10
 - 31s - loss: 0.0085 - acc: 0.9974 - val_loss: 1.0567 - val_acc: 0.8052
Epoch 8/10
 - 32s - loss: 0.0146 - acc: 0.9942 - val_loss: 1.0566 - val_acc: 0.8094
Epoch 9/10
 - 29s - loss: 0.0224 - acc: 0.9923 - val_loss: 1.1312 - val_acc: 0.7972
Epoch 10/10
 - 28s - loss: 0.0203 - acc: 0.9925 - val_loss: 1.1651 - val_acc: 0.7896


In [77]:
scores = model.evaluate(x_test, y_test, verbose=1)
scores[1]

25000/25000 [==============================] - 7s 278us/step


0.84748

In [22]:
predict = model.predict_classes(x_test)

In [24]:
predict[0:10]

array([[0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]], dtype=int32)

In [25]:
predict_classes = predict.reshape(-1)
predict_classes[:10]

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [26]:
SentimentDict = {1:'正面的', 0:'負面的'}
def display_test_Sentiment(i):
    print(test_text[i])
    print('label真實值：', SentimentDict[y_test[i]], '預測結果：', SentimentDict[predict_classes[i]])

In [27]:
display_test_Sentiment(2)

How I Spend My Vacation puts closure to the television series that aired from 1990 to 1992. I've always enjoyed the series for what it is. I've never compared the series to the Looney Tunes of the old days. The video release was split up in four or five episodes (I can't remember as I'm writing this) for television and it's included in the episode list of the series. What's good about this series is that All the main characters of the show have a major role in the story. We see what they all do during their summer vacation in different parts of the world. Elmyra is with his parents in a Safari, Plucky and Hampton crossing the country with Hampton's parents, Fifi is in some beach, etc. The climax is very good and at the end all Tiny Toon characters reunite for the start of a new year in the "Looniversity", thus practically ending the good run of this tv show.
label真實值： 正面的 預測結果： 正面的


In [48]:
input_text="I had the opportunity to watch this movie today and man it was disappointing. It was boring. reason being perhaps that it felt like the copy of the original. These were the great cartoons which we grew up watching. But why would the same songs, characters, story be entertaining for us, we could just watch the cartoons again, why bother to remake. It might have been entertaining if characters were a little believable at least. The beast lack any emotion. Emma Watson (though I love her) was just trying to copy belle from the cartoons. She wasn't even given a chance to try to portray the character of belle and not the 'acting' of cartoon belle (she wasn't even successful in that). The only person I liked in the movie was Gaston. The rest of the characters were soooo animated that it stole the real feel of the movie which should really be the purpose of remaking a movie after all. A few new songs were added which were quite boring. The village of belle was aweeeefuulllll. Why Why Why would they make a village which just look like a ground full of hideous people. Why not put a little more money in that part. The reason of the movie should be to give a real life feeling to fairy tales ( which jungle book and Cinderella gave), not to just make it more cartoonish."

In [49]:
input_seq = token.texts_to_sequences([input_text])

In [50]:
print(input_seq)

[[9, 65, 1, 1428, 5, 102, 10, 16, 636, 2, 128, 8, 12, 1332, 8, 12, 354, 279, 108, 377, 11, 8, 417, 36, 1, 1034, 4, 1, 200, 130, 67, 1, 83, 59, 71, 52, 145, 17, 134, 58, 1, 168, 686, 101, 61, 25, 438, 14, 174, 71, 96, 39, 102, 1, 170, 134, 1416, 5, 1029, 8, 234, 24, 73, 438, 44, 101, 67, 3, 113, 860, 29, 218, 1, 579, 97, 1421, 147, 9, 115, 37, 12, 39, 265, 5, 1034, 35, 1, 55, 281, 56, 344, 3, 577, 5, 349, 5, 1971, 1, 105, 4, 2, 20, 1, 4, 1068, 55, 281, 56, 1107, 7, 11, 1, 60, 411, 9, 420, 7, 1, 16, 12, 1, 356, 4, 1, 101, 67, 1120, 11, 8, 1, 143, 230, 4, 1, 16, 59, 140, 62, 25, 1, 1285, 4, 3, 16, 99, 28, 3, 167, 157, 686, 67, 1281, 59, 67, 175, 354, 1, 4, 12, 134, 134, 134, 58, 32, 93, 3, 59, 39, 164, 36, 3, 1560, 364, 4, 80, 134, 20, 272, 3, 113, 49, 274, 7, 11, 169, 1, 279, 4, 1, 16, 140, 25, 5, 198, 3, 143, 109, 543, 5, 59, 270, 2, 515, 20, 5, 39, 93, 8, 49]]


In [51]:
len(input_seq[0])

209

In [52]:
pad_input_seq = sequence.pad_sequences(input_seq, maxlen=100)

In [54]:
len(pad_input_seq[0])

100

In [55]:
predict_result = model.predict_classes(pad_input_seq)

In [56]:
predict_result

array([[0]], dtype=int32)

In [57]:
predict_result[0][0]

0

In [58]:
SentimentDict[predict_result[0][0]]

'負面的'

In [59]:
def predict_review(input_text):
    input_seq = token.texts_to_sequences([input_text])
    pad_input_seq = sequence.pad_sequences(input_seq, maxlen=100)
    predict_result = model.predict_classes(pad_input_seq)
    print(SentimentDict[predict_result[0][0]])

In [62]:
predict_review("A selfish prince (Dan Stevens) lords over the local villagers. One rainy night, an old woman seeking shelter is sent away. She turns out to be an Enchantress. She curses the prince and his servants. He's turned into a Beast to be forgotten by the world. Belle (Emma Watson) is tired of the provincial life and the tiresome pursuing self-obsessed Gaston (Luke Evans). LeFou (Josh Gad) is his adoring man-servant. Belle asks for a rose from her traveling father. He takes a rose from the castle and is imprisoned by the Beast. Belle volunteers to take her father's place as the Beast's prisoner.Emma Watson is a good actress. Her singing is average but that has its charms. Gaston is a great villain. The CGI looks good for the most part but I am uncertain about the Beast. Belle is a modern empowered character which does strike me as a little odd. It would help to have more scenes like the villager berating Belle for teaching the little girl to read. The more backwards the villagers, the better it is for Belle to rebel against. Then there is the basic prisoner love story at the core of this fairy tale. It remains problematic to me. There are ways to ameliorate this but the movie sticks to the story. This is good but the Disney animation remains the better.")

正面的
